# Rappel de l'objectif

<span style="color:red">L'objectif est de répartir un budget donné sur un intervalle de temps donné.</span>


```Difficultés``` : 
- Ne pas épuiser le budget avant la fin de l'intervalle
- Ne pas avoir de vide de dépense durant l'intervalle
- Il ne doit pas rester de budget à la fin de l'intervalle

In [1]:
import simpy
import random

## 1. Simulation totalement manuelle (sans variables)

Le cadre est le suivant : 
- Une bid request toute les minutes pendant une heure
- On bid une valeur entre 0 et 10€
- Si on est au dessus de 5€ on gagne et on paye le bid, sinon on perd et on ne paye rien

In [2]:
time_between_br = 1

In [3]:
def bidding(env):
    budget = 100
    while True:
        print(f'Bid request at {env.now}')
        bid = random.random()
        bid = round(bid, 2)
        if bid > 0.5 and budget > 0:
            print("\t Win")
            print(f"\t Paid {bid*10}")
            budget = budget - bid*10 
        else:
            print("\t Lose")
            print("\t Paid 0")
        print(f'\t Remaining budget is {budget} \n')
        yield env.timeout(time_between_br)

In [4]:
env = simpy.Environment()

In [5]:
env.process(bidding(env))

<Process(bidding) object at 0x7f21882da4d0>

In [7]:
env.run(until=60)

Bid request at 0
	 Lose
	 Paid 0
	 Remaining budget is 100 

Bid request at 1
	 Win
	 Paid 9.8
	 Remaining budget is 90.2 

Bid request at 2
	 Lose
	 Paid 0
	 Remaining budget is 90.2 

Bid request at 3
	 Win
	 Paid 5.699999999999999
	 Remaining budget is 84.5 

Bid request at 4
	 Lose
	 Paid 0
	 Remaining budget is 84.5 

Bid request at 5
	 Win
	 Paid 6.5
	 Remaining budget is 78.0 

Bid request at 6
	 Lose
	 Paid 0
	 Remaining budget is 78.0 

Bid request at 7
	 Lose
	 Paid 0
	 Remaining budget is 78.0 

Bid request at 8
	 Lose
	 Paid 0
	 Remaining budget is 78.0 

Bid request at 9
	 Lose
	 Paid 0
	 Remaining budget is 78.0 

Bid request at 10
	 Win
	 Paid 6.0
	 Remaining budget is 72.0 

Bid request at 11
	 Win
	 Paid 7.4
	 Remaining budget is 64.6 

Bid request at 12
	 Lose
	 Paid 0
	 Remaining budget is 64.6 

Bid request at 13
	 Win
	 Paid 8.9
	 Remaining budget is 55.699999999999996 

Bid request at 14
	 Win
	 Paid 7.0
	 Remaining budget is 48.699999999999996 

Bid request at 15

On a ici simulé ce qui aurait pu se passer sur une heure donnée. Dans cette simulation le budget est dépensé beaucoup trop rapidement dans l'heure. L'objectif serait de simuler une situation dans laquelle le budget serait dépensé uniformément dans l'heure.

L'idée est la suivante : la probabilité de gagner dépend désormais du montant de la bid request. On estime le nombre de br qui va venir dans l'heure (dans notre cas l'estimation est exacte). Puis selon le budget total on attribue un budget par bid request. Si on gagne, alors tout se passe bien, si on perd alors l'idée et de faire augmenter la proba de gagner pour la prochaine bid request en ajoutant le budget non alloué de la bid request perdue à la prochaine. 

In [37]:
def win_proba(p):
    return random.random() < p

In [53]:
def bidding(env):
    total_budget = 60
    estimate_br = 60
    budget_per_br = total_budget/estimate_br
    proba = 0.5
    budget = total_budget
    while True:
        print(f'Bid request at {env.now}')
        bid = budget_per_br
        if win_proba(proba) and budget > 0:
            print("\t Win")
            print(f"\t Paid {bid}")
            budget = budget - bid
            budget_per_br = total_budget/estimate_br
            proba = 0.5
        else:
            print("\t Lose")
            print("\t Paid 0")
            budget_per_br += total_budget/estimate_br
            proba += 0.1
        print(f'\t Remaining budget is {budget} \n')
        yield env.timeout(time_between_br)

In [61]:
env = simpy.Environment()
env.process(bidding(env))
env.run(until=60)

Bid request at 0
	 Win
	 Paid 1.0
	 Remaining budget is 59.0 

Bid request at 1
	 Lose
	 Paid 0
	 Remaining budget is 59.0 

Bid request at 2
	 Win
	 Paid 2.0
	 Remaining budget is 57.0 

Bid request at 3
	 Lose
	 Paid 0
	 Remaining budget is 57.0 

Bid request at 4
	 Lose
	 Paid 0
	 Remaining budget is 57.0 

Bid request at 5
	 Lose
	 Paid 0
	 Remaining budget is 57.0 

Bid request at 6
	 Win
	 Paid 4.0
	 Remaining budget is 53.0 

Bid request at 7
	 Win
	 Paid 1.0
	 Remaining budget is 52.0 

Bid request at 8
	 Lose
	 Paid 0
	 Remaining budget is 52.0 

Bid request at 9
	 Win
	 Paid 2.0
	 Remaining budget is 50.0 

Bid request at 10
	 Win
	 Paid 1.0
	 Remaining budget is 49.0 

Bid request at 11
	 Win
	 Paid 1.0
	 Remaining budget is 48.0 

Bid request at 12
	 Win
	 Paid 1.0
	 Remaining budget is 47.0 

Bid request at 13
	 Win
	 Paid 1.0
	 Remaining budget is 46.0 

Bid request at 14
	 Win
	 Paid 1.0
	 Remaining budget is 45.0 

Bid request at 15
	 Lose
	 Paid 0
	 Remaining budget is